In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install ftfy accelerate transformers
!pip install git+https://github.com/openai/CLIP.git

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import clip
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
class Config:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    clip_model_name = "ViT-B/32"
    batch_size = 16
    lr = 1e-5
    epochs = 15
    real_dir = "/content/drive/MyDrive/dataset/dataset1/real"
    fake_dir = "/content/drive/MyDrive/dataset/dataset1/forge"
    question = "Is this signature authentic or forged?"

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

class SignatureDataset(Dataset):
    def __init__(self, question):
        self.real_images = [os.path.join(Config.real_dir, f) for f in os.listdir(Config.real_dir)]
        self.fake_images = [os.path.join(Config.fake_dir, f) for f in os.listdir(Config.fake_dir)]
        self.all_images = self.real_images + self.fake_images
        self.labels = [0]*len(self.real_images) + [1]*len(self.fake_images)
        self.question = question
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        image = Image.open(self.all_images[idx]).convert("RGB")
        image = transform(image)

        text_ids = self.tokenizer.encode(self.question, return_tensors="pt").squeeze(0)
        max_length = 77
        if text_ids.size(0) < max_length:
            padding = torch.zeros(max_length - text_ids.size(0), dtype=torch.long)
            text_ids = torch.cat([text_ids, padding])
        else:
            text_ids = text_ids[:max_length]

        label = self.labels[idx]
        return image, text_ids, label

In [ ]:
class ExplainableForgeryDetector(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.device = Config.device
        self.clip_model, _ = clip.load(Config.clip_model_name, device=self.device)
        self.classifier = torch.nn.Linear(512, 1)

        self.explainer = GPT2LMHeadModel.from_pretrained("gpt2-medium").to(self.device)
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")

    def forward(self, images, questions):
        images = images.to(self.device)
        questions = questions.to(self.device)

        with torch.no_grad():
            image_features = self.clip_model.encode_image(images)

        logits = self.classifier(image_features).squeeze(-1)

        decoded_question = self.tokenizer.decode(questions[0], skip_special_tokens=True)
        input_ids = self.tokenizer.encode(decoded_question, return_tensors="pt").to(self.device)
        explanation_ids = self.explainer.generate(input_ids, max_length=50, num_beams=3, no_repeat_ngram_size=2)
        explanation = self.tokenizer.decode(explanation_ids[0], skip_special_tokens=True)

        return logits, explanation

In [ ]:
dataset = SignatureDataset(Config.question)
dataloader = DataLoader(dataset, batch_size=Config.batch_size, shuffle=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

In [ ]:
model = ExplainableForgeryDetector().to(Config.device)
optimizer = torch.optim.AdamW([
    {'params': model.classifier.parameters()},
    {'params': model.explainer.parameters(), 'lr': 1e-6}
], lr=Config.lr)
criterion = torch.nn.BCEWithLogitsLoss()

100%|███████████████████████████████████████| 338M/338M [00:08<00:00, 41.2MiB/s]
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
for epoch in range(Config.epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for images, questions, labels in dataloader:
        images = images.to(Config.device)
        questions = questions.to(Config.device)
        labels = labels.float().to(Config.device)

        optimizer.zero_grad()
        logits, explanation = model(images, questions)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).float()
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_samples  # Calculate epoch accuracy
    print(f"Epoch {epoch+1}/{Config.epochs} - Loss: {avg_loss:.4f} - Accuracy: {accuracy:.4f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Epoch 1/15 - Loss: 0.5985 - Accuracy: 0.7778


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Epoch 2/15 - Loss: 0.5944 - Accuracy: 0.7778


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

...

Epoch 13/15 - Loss: 0.5709 - Accuracy: 0.7778


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Epoch 14/15 - Loss: 0.5695 - Accuracy: 0.7778


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Epoch 15/15 - Loss: 0.5661 - Accuracy: 0.7778


In [ ]:
def predict_and_explain(image_path):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(Config.device)

    question_ids = model.tokenizer.encode(Config.question, return_tensors="pt").squeeze(0)
    max_length = 77
    if question_ids.size(0) < max_length:
        padding = torch.zeros(max_length - question_ids.size(0), dtype=torch.long)
        question_ids = torch.cat([question_ids, padding])
    else:
        question_ids = question_ids[:max_length]
    question_ids = question_ids.unsqueeze(0).to(Config.device)

    with torch.no_grad():
        logits, explanation = model(image_tensor, question_ids)
        pred_label = "Authentic" if torch.sigmoid(logits).item() < 0.5 else "Forged"

    return pred_label, explanation

# Example :
test_image_path = "/content/drive/MyDrive/dataset/dataset1/forge/02100003.png"
prediction, explanation = predict_and_explain(test_image_path)
print(f"Prediction: {prediction}")
print(f"Explanation: {explanation}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prediction: Forged
Explanation: Is this signature authentic or forged?!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

This signature is not authentic. It is a forged signature. Please verify the signature by clicking on the link below.


https://www.facebook.
